<a href="https://colab.research.google.com/github/varunsboy17/Credit-Card-Fraud-Analysis-Prediction-using-AutoML/blob/master/CreditCardFraud_with_H2OML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
import pandas as pd

!pip install h2o
import h2o

In [17]:
print(h2o.__version__)
from h2o.automl import H2OAutoML
h2o.init(max_mem_size = '16G')

3.26.0.4
Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,27 mins 31 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.4
H2O cluster version age:,1 day
H2O cluster name:,H2O_from_python_unknownUser_y8tcaq
H2O cluster total nodes:,1
H2O cluster free memory:,13.84 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
%%time
train_trns = h2o.import_file('/content/drive/My Drive/Credit Card Fraud Dataset/train_transaction.csv')
train_id = h2o.import_file('/content/drive/My Drive/Credit Card Fraud Dataset/train_identity.csv')
test_trns = h2o.import_file('/content/drive/My Drive/Credit Card Fraud Dataset/test_transaction.csv')
test_id = h2o.import_file('/content/drive/My Drive/Credit Card Fraud Dataset/test_identity.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
CPU times: user 1.23 s, sys: 147 ms, total: 1.38 s
Wall time: 1min 3s


In [0]:
all_train = train_trns.merge(train_id, all_x=True)
all_test = test_trns.merge(test_id, all_x=True)

In [0]:
x = all_train.drop('isFraud').columns
y = 'isFraud'
all_train[y] = all_train[y].asfactor()

In [30]:
aml = H2OAutoML(max_models=30, seed=45, max_runtime_secs=7200)
aml.train(x=x, y=y, training_frame=all_train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [31]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_1_AutoML_20190914_074629,0.825778,0.301448,0.287349,0.317855,0.101032


In [32]:
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20190914_074629


Model Summary: 

,,number_of_trees
0,,92.0




ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.01627403606714754
RMSE: 0.12756973021507703
LogLoss: 0.07260630984218717
Mean Per-Class Error: 0.13040002343787382
AUC: 0.9394105792978894
pr_auc: 0.7329075195310224
Gini: 0.8788211585957788

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.21922570363334987: 

,,0,1,Error,Rate
0,0,567281.0,2596.0,0.0046,(2596.0/569877.0)
1,1,8055.0,12608.0,0.3898,(8055.0/20663.0)
2,Total,575336.0,15204.0,0.018,(10651.0/590540.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


Scoring History: 

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2019-09-14 09:26:46,1:40:16.834,0.0,0.500000,0.693147,0.500000,0.000000,1.000000,0.965010
1,,2019-09-14 09:28:05,1:41:35.791,5.0,0.399020,0.508855,0.861102,0.514904,27.405083,0.025451
2,,2019-09-14 09:28:28,1:41:58.689,10.0,0.323659,0.389058,0.869087,0.555802,27.587576,0.024870
3,,2019-09-14 09:28:54,1:42:24.570,15.0,0.267923,0.306608,0.875412,0.558451,27.752105,0.024246
4,,2019-09-14 09:29:20,1:42:50.632,20.0,0.227327,0.247906,0.878577,0.585050,27.863403,0.023721
5,,2019-09-14 09:29:45,1:43:15.605,25.0,0.198176,0.205046,0.884303,0.602719,27.965024,0.023082
6,,2019-09-14 09:30:12,1:43:41.982,30.0,0.177609,0.173215,0.889112,0.616931,28.076323,0.022561
7,,2019-09-14 09:30:40,1:44:10.280,35.0,0.163472,0.149504,0.892880,0.626135,28.148909,0.022068
8,,2019-09-14 09:31:09,1:44:38.897,40.0,0.153728,0.131569,0.897692,0.638609,28.206978,0.022026
9,,2019-09-14 09:31:38,1:45:08.729,45.0,0.147095,0.118025,0.901023,0.649975,28.279564,0.022088



Variable Importances: 

,variable,relative_importance,scaled_importance,percentage
0,V258,52436.750000,1.000000,0.100520
1,C1,24817.818359,0.473291,0.047575
2,C13,21560.853516,0.411178,0.041332
3,C14,21370.853516,0.407555,0.040968
4,V294,14610.700195,0.278635,0.028009
5,V201,13075.166016,0.249351,0.025065
6,TransactionAmt,11105.234375,0.211783,0.021289
7,C11,9712.950195,0.185232,0.018620
8,TransactionID,9595.095703,0.182984,0.018394
9,D2,8880.705078,0.169360,0.017024



See the whole table with table.as_data_frame()


In [0]:
h2o.remove(train_trns)
h2o.remove(train_id)
h2o.remove(test_trns)
h2o.remove(test_id)
#test = h2o.import_file('/content/drive/My Drive/Credit Card Fraud Dataset/test_transaction.csv')

In [37]:
preds = aml.predict(all_test)
preds['p1'].as_data_frame().values.flatten().shape

xgboost prediction progress: |████████████████████████████████████████████| 100%


/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'DeviceInfo' has levels not trained on: [2014819, 2PS64, 4009A, 4027A, 4047A, 4047G, 5010G, 5011A, 5012G, 5015A, 5025G, 5026A, 5026A Build/NRD90M, 5033A, 5041C, 5044T, 5049W, 5049Z, 5052Y, 5054S, 5056A, 5058A, 5059A, 5080A, 5080X, 5085A, 5085B, 5086A, 5099A, 5099A Build/O00623, 6016A, 6036A Build/JLS36C, 6043A, 6045O, 6062W, 7040N, 7048A, 7048X, 8050G, 8080, 8082, 8_Plus, 9003A, 9024W, A10, A10 Pro Build/LMY47I, A1_PRO, A37fw, A462C, A502DL, A50LT, A622GL, A7, AGS-L03, AGS-L09, AGS-W09, ALCATELONETOUCH, ALE-L02, ALE-L21, ALE-L23, ALP-L09, ALP-L29, ANE-LX1, ANE-LX1 Build/HUAWEIANE-LX1, ANE-LX2, ANE-LX3, ASUS_A002A, ASUS_A006, ASUS_A007, ASUS_A009, ASUS_X008DC, ASUS_X00AD, ASUS_X00HD, ASUS_X00LDA, ASUS_X00PD, ASUS_Z00XS, ASUS_Z012DC, ASUS_Z017DC, ASUS_Z01HD, ASUS_Z01MD, ATU-L11, ATU-L21, ATU-LX3, ATU-LX3 Build/HUAWEIATU-LX3, AUM-L29, AW790, AX681, AX921, Alcatel_5044R, Alcatel_5098O, Alcatel

(506691,)

In [39]:
sample_submission = pd.read_csv('/content/drive/My Drive/Credit Card Fraud Dataset/sample_submission.csv')
sample_submission.shape

(506691, 2)

In [0]:
sample_submission['isFraud'] = preds['p1'].as_data_frame().values
sample_submission.to_csv('h2o_sample_submission.csv', index=False)